In [1]:
from project_lib import *
import tqdm

In [2]:
df = pd.read_csv('dataset_tmc.csv', parse_dates = ['Dates'], date_format = '%d/%m/%y')
df.columns = [i.lower() for i in df.columns]
df.columns = [i.replace(' ','_') for i in df.columns]
df = df.set_index('dates')

# Check NaNs
if df.isna().sum().sum() != 0:
    print('check NaNs in data')

# Drop Sat&Sun if any
df = df[df.index.dayofweek<5]

In [3]:
# STRATEGY PARAMETERS
TRADED_SECURITIES = [
            'us_staples',
                'eu_healthcare', 'japan', 'brazil', 'nasdaq', 'us_2y',
                'us_15y', 'germany_10y', 'uk_15y', 'us_hy', 'oil', 'ind_metals', 'agri',
                'gold', 'silver', 'eurusd', 'usdjpy', 'chfjpy', 'eurbrl', 'gbpmxn'
    ]

LONG_PARAMS = {10:0.05, 21:0.15, 63:0.25} # dict with lenght of window as key and percentile for longs as values
SHORT_PARAMS = {10:0.95, 21:0.85, 63:0.75} # dict with lenght of window as key and percentile for shorts as values
MA_FAST_WDW = 20
MA_SLOW_WDW = 60

SUPPORTIVE_PCTL_MOVE = 0.3
COUNTER_PCTL_MOVE = 0.2
TRADES_MAX_DAYS = 21

MAX_DOLLAR_LOSS = 100


In [4]:
# list to store all trades
trades_list = []
trades_pnl = {}
portfolio_pnl = [] ##### <-------------------------------------------- DROP

for security_id in tqdm.tqdm(TRADED_SECURITIES):
    # generate signal based on standard strategy
    signal_all = meanrev_signal(df[security_id],
                                long_params = LONG_PARAMS,
                                short_params = SHORT_PARAMS,
                                ma_fast_wdw = MA_FAST_WDW,
                                ma_slow_wdw = MA_SLOW_WDW
                )

    # "start your backtest at t-10". hard coding initial date for BT
    signal_all = signal_all.loc['2014-02-12':]

    # extract from all signal only the actual open buy/sell triggers
    signal_do = signal_all[signal_all!= 0]

    # iterates over all buy/sell signal and execute orders
    for dt_open, direction in zip(signal_do.index, signal_do):
        
        # create a unique id for each trade
        trade_id = security_id+'#'+str(dt_open)[:10]

        # price at which the trade is open
        price_open = df.loc[dt_open, security_id]

        # compute TP/SL returns and prices
        tp_return, sl_return = tp_sl_rule(df[security_id],
                                            dt_open,
                                            direction,
                                            supportive_pctl_move = SUPPORTIVE_PCTL_MOVE,
                                            counter_pctl_move = COUNTER_PCTL_MOVE
                                          )
        price_tp = price_open * (1 + tp_return*direction)
        price_sl = price_open * (1 + sl_return*direction)

        # compute the optimal sizing such that all trades loses the same amount of $ if SL is hitted
        quantity = MAX_DOLLAR_LOSS/((price_open - price_sl)*direction)

        # store all trade info in df
        trade = pd.DataFrame({'security_id':security_id,
                                'dt_open':str(dt_open)[:10],
                                'price_open':price_open,
                                'direction':direction,
                                'quantity':quantity,
                                'price_tp':price_tp,
                                'price_sl':price_sl}, index=[trade_id])

        # create a temporary df that contains prices of instrument during trade
        dt_open_idxdf = list(df.index).index(dt_open)
        temp_px = df[[security_id]].iloc[dt_open_idxdf + 1: dt_open_idxdf + TRADES_MAX_DAYS + 1]
        temp_px['price_open'] = price_open
        temp_px['direction'] = direction
        temp_px['quantity'] = quantity
        temp_px['price_tp'] = price_tp
        temp_px['price_sl'] = price_sl

        # Check if and when a TP/SL is triggered and cut the temporary df accordingly
        if direction==1:
            temp_px['tp_hitted'] = (temp_px[security_id] > temp_px['price_tp']) * 1
            temp_px['sl_hitted'] = (temp_px[security_id] < temp_px['price_sl']) * 1
        elif direction==-1:
            temp_px['tp_hitted'] = (temp_px[security_id] < temp_px['price_tp']) * 1
            temp_px['sl_hitted'] = (temp_px[security_id] > temp_px['price_sl']) * 1

        temp_px['tp_sl_hitted'] = temp_px['tp_hitted'] + temp_px['sl_hitted']

        if 1 in list(temp_px['tp_sl_hitted']):
            dt_close = str(temp_px[temp_px['tp_sl_hitted']==1].index[:1][0])[:10]
            exit_type = 'TP/SL exit'
        else:
            dt_close = str(list(temp_px.index)[-1:][0])[:10]
            exit_type = 'max duration'

        # cut temp_px at closing date
        temp_px = temp_px.loc[:dt_close]
        # compute PnL of the trade during the days it was open and append to Portfolio PnL list
        temp_px['pnl'] = temp_px[security_id] * temp_px['quantity']
        temp_px['daily_return'] = temp_px[security_id].pct_change().fillna(
            temp_px[security_id].iloc[0]/temp_px['price_open'].iloc[0]-1)
        
        portfolio_pnl.append(temp_px['daily_return'].rename(trade_id)) ##### <-------------------------------------------- DROP

        # store closing price of the trade
        price_close = temp_px.loc[dt_close, security_id]

        # add closing trade date and price to trade df
        trade['dt_close'] = dt_close
        trade['price_close'] = price_close
        trade['duration'] = len(temp_px)
        trade['exit_condition'] = exit_type

        # compute the annualized volatility of the trade
        trade['ann_volatility'] = temp_px['daily_return'].std() * np.sqrt(252)

        # collect trade and pnl
        trades_list.append(trade)
        trades_pnl[trade_id] = temp_px

# check if trades_list is non-empty
if len(trades_list)==0:
    raise Exception('No trades have been executed')

trades_list = pd.concat(trades_list)
# compute trades' returns
trades_list['return'] = trade_return(trades_list)
trades_list['daily_return'] = trades_list['return'] / trades_list['duration']
trades_list['sharpe_ratio'] = trades_list['daily_return']*252 / trades_list['ann_volatility']
# split between TP and SL exit explicitly
trades_list['exit_condition'] = trades_list['exit_condition'].mask(
    (trades_list['exit_condition']=='TP/SL exit')&(trades_list['return']>0), 'TP exit')
trades_list['exit_condition'] = trades_list['exit_condition'].mask(
    (trades_list['exit_condition']=='TP/SL exit')&(trades_list['return']<0), 'SL exit')

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:46<00:00,  2.31s/it]


In [5]:
# trades_list.groupby('security_id').apply(lambda x: hit_ratio(x), include_groups=False).rename('hit_ratio').to_frame().join(
#     trades_list.groupby('security_id').apply(lambda x: win_loss(x), include_groups=False).rename('win_loss').to_frame())

In [6]:
# violin_plot_grouped(trades_list, 'security_id', 'daily_return')
# violin_plot_grouped(trades_list.sort_values(by='duration'), 'duration', 'daily_return')
# violin_plot_grouped(trades_list, 'direction', 'daily_return', figsize=(8,5))
# plot_histogram(trades_list['daily_return'], 'Distribution of daily returns from all trades', figsize=(20, 8), bins=100)
# trades_list.groupby('security_id')['daily_return'].describe()

In [8]:
trades_list.groupby('security_id')['sharpe_ratio'].mean()

security_id
agri               0.599256
brazil            -5.904272
chfjpy             6.138262
eu_healthcare     -3.051147
eurbrl            22.528448
eurusd             7.805235
gbpmxn             1.860837
germany_10y        4.486594
gold              -6.945390
ind_metals        -4.086026
japan             -0.539324
nasdaq            -2.749089
oil               -8.197474
silver            18.398421
uk_15y             1.233333
us_15y             7.034130
us_2y            380.314556
us_hy            -19.037939
us_staples        -6.455928
usdjpy            10.111792
Name: sharpe_ratio, dtype: float64

In [11]:
trades_list

,security_id,dt_open,price_open,direction,quantity,price_tp,price_sl,ann_volatility,dt_close,price_close,duration,exit_condition,return,daily_return,sharpe_ratio
us_staples#2014-03-04,us_staples,2014-03-04,42.6900,-1,202.680699,42.213262,43.183387,0.062565,2014-04-02,43.0400,21,max duration,-0.008199,-0.000390,-1.572493
us_staples#2014-03-10,us_staples,2014-03-10,42.7000,-1,195.314722,42.223150,43.211994,0.064128,2014-04-08,43.2100,21,max duration,-0.011944,-0.000569,-2.234995
us_staples#2014-03-12,us_staples,2014-03-12,42.7700,-1,194.662956,42.292368,43.283708,0.066856,2014-04-09,43.4000,20,SL exit,-0.014730,-0.000736,-2.776071
us_staples#2014-03-17,us_staples,2014-03-17,42.8300,-1,193.542600,42.351698,43.346682,0.066247,2014-04-09,43.4000,17,SL exit,-0.013308,-0.000783,-2.977929
us_staples#2014-03-18,us_staples,2014-03-18,42.9200,-1,192.623436,42.440693,43.439148,0.084037,2014-04-16,43.6200,21,SL exit,-0.016309,-0.000777,-2.328885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
gbpmxn#2024-01-18,gbpmxn,2024-01-18,21.8110,-1,686.960469,21.553722,21.956569,0.075808,2024-02-05,21.4659,12,TP exit,0.015822,0.001319,4.383039
gbpmxn#2024-01-22,gbpmxn,2024-01-22,21.8408,-1,688.438444,21.583171,21.986056,0.069400,2024-02-05,21.4659,10,TP exit,0.017165,0.001717,6.232832
gbpmxn#2024-01-23,gbpmxn,2024-01-23,21.9541,-1,684.082758,21.695134,22.100281,0.052833,2024-02-02,21.6512,8,TP exit,0.013797,0.001725,8.226047
gbpmxn#2024-02-05,gbpmxn,2024-02-05,21.4659,1,639.280391,21.690898,21.309474,0.050552,2024-02-12,21.5455,5,max duration,0.003708,0.000742,3.697063


In [32]:
252**(1/2)

15.874507866387544

In [30]:
np.sqrt(252)

15.874507866387544

In [20]:
# controlla se ha senso

gg = trades_list.groupby('security_id').apply(lambda x: x.assign(pnl_cumul = x['pnl'].cumsum())).reset_index(drop=True).sort_values(by=['security_id', 'dt_open'])

KeyError: 'pnl'

In [13]:
gg[gg['security_id'] == 'brazil']['pnl_cumul'].plot()

NameError: name 'vgg' is not defined

### cHeCks
using a rolling window on the price series of respectively 10, 21, 63d I compute the 5th, 15th, 25th percentiles of the distribution and I check, for the Long signal 1, if today's Price is below ALL the 5th, 15th, 25th percentiles.


In [27]:
prova = df[['us_staples']].copy()
prova['q5_2w'] = prova['us_staples'].rolling(10).quantile(0.05)
prova['q15_1m'] = prova['us_staples'].rolling(21).quantile(0.15)
prova['q25_3m'] = prova['us_staples'].rolling(63).quantile(0.25)
prova['ma20'] = prova['us_staples'].rolling(20).mean()
prova['ma60'] = prova['us_staples'].rolling(60).mean()
prova = prova.loc['2014-02-12':].copy()

In [28]:
prova['s1_long'] = 1 * (
    (prova['us_staples'] < prova['q5_2w']) & (prova['us_staples'] < prova['q15_1m']) & (prova['us_staples'] < prova['q25_3m']))
prova['s2_long'] = 1 * ((prova['us_staples'] < prova['ma20']) & (prova['us_staples'] < prova['ma60']))

prova['long'] = (prova['s1_long']*prova['s2_long'])

In [29]:
signal_all[signal_all==1]

dates
2014-07-29    1
2014-07-30    1
2014-07-31    1
2014-10-15    1
2014-10-16    1
             ..
2023-10-03    1
2023-10-05    1
2023-10-06    1
2023-10-12    1
2023-10-27    1
Length: 166, dtype: int64

In [37]:
prova[prova['s1_long']==1]['s2_long'].unique()

array([1])